In [1]:
%load_ext autoreload
%autoreload 2

In [80]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
from fastai2.vision.all import load_image, get_image_files, show_image, \
    show_images
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from pathlib import Path
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from htools import *
from img_wang.torch_utils import *

In [15]:
cd_root()
gpu_setup()

Current directory: /storage/img-wang
Setting seeds for reproducible training.


In [101]:
class AlbumentationsDataset(Dataset):

    @valuecheck
    def __init__(self, dir_=None, paths=(), shape=(128, 128), pct_pos=.5,
                 tfm='ChannelShuffle', **tfm_kwargs):
        if not dir_ and not paths:
            raise ValueError('One of dir_ or paths should be non-null.')

        self.paths = paths or get_image_files(dir_)
        self.shape = shape
        self.pct_pos = pct_pos
        self.load_img = load_image
        self.tfm = getattr(A, tfm)(always_apply=True, **tfm_kwargs)
        self.tfm_pipeline = A.Compose([A.Resize(*shape),
                                       ToTensorV2()])

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, i):
        """
        1 if patch is from the original image, 0 otherwise.
        """
        img = np.array(self.load_img(self.paths[i]))
        if len(img.shape) == 2: img = np.repeat(x[None, ...], 3, axis=0)
        y = 0
        if np.random.uniform() < self.pct_pos:
            img = self.tfm(image=img)['image']
            y = 1
        return self.tfm_pipeline(image=img)['image']/255.0, y

In [102]:
paths = get_image_files('data/imagewang-160/unsup')[:32]

In [103]:
ds = AlbumentationsDataset(paths=paths)
dl = DataLoader(ds, batch_size=4, shuffle=True)

In [110]:
xb.shape

torch.Size([4, 3, 128, 128])

In [122]:
for i, (xb, yb) in enumerate(dl):
    xb = [x for x in xb.cpu()]
    print([x.shape for x in xb])
#     show_images(xb, titles=yb.cpu().numpy().tolist())
    if i > 2: break

[torch.Size([3, 128, 128]), torch.Size([3, 128, 128]), torch.Size([3, 128, 128]), torch.Size([3, 128, 128])]


error: OpenCV(4.4.0) /tmp/pip-req-build-2mquh264/opencv/modules/imgproc/src/resize.cpp:3807: error: (-215:Assertion failed) func != 0 in function 'resize'
